# Butterworth Filter example

![Butterworth Filter](./pictures/butterworth_dp.svg)

In [ ]:
from ahkab import new_ac, new_op, new_tran, run
from ahkab.circuit import Circuit
from ahkab.plotting import plot_results # calls matplotlib for you
import numpy as np


# Define the circuit
cir = Circuit(title="Butterworth Low-pass filter")

gnd = cir.get_ground_node()

cir.add_vsource('V1', 'n1', cir.gnd, dc_value=0., ac_value=1.)
cir.add_resistor("R1", n1="n1", n2="n2", value=600)
cir.add_inductor("L1", n1="n2", n2="n3", value=15.24e-3)
cir.add_capacitor("C1", n1="n3", n2=gnd, value=119.37e-9)
cir.add_inductor("L2", n1="n3", n2="n4", value=61.86e-3)
cir.add_capacitor("C2", n1="n4", n2=gnd, value=155.12e-9)
cir.add_resistor("R2", n1="n4", n2=gnd, value=1.2e3)

# Define the analysis
ac1 = new_ac(1e3, 1.0e4, 1e2, x0=None)

# run it
res = run(cir, ac1)

# plot the results
plot_results('3rd order 1kHz Butterworth LP-filter', [('|Vn4|',"")], res['ac'],
             outfilename='plots/bpf_transfer_fn.png')

print(cir)

In [ ]:
ckt_file = open('./ckts/butterworth_lp.ckt', 'w+')
print(cir, file=ckt_file)
print( "\n.tran tstop=150n tstep=.1n method=trap uic=2\n", file=ckt_file)
print( "\n.ac start=1e3 stop=1.0e4 nsteps=200\n", file=ckt_file)
# ckt = sprint(cir)
# ckt_file = open('./ckts/butterworth.ckt', 'w+')
# ckt_file.write(cir)
ckt_file.close()

## Run simulation from file

In [ ]:
from ahkab.netlist_parser import parse_circuit, parse_analysis
import tempfile

(circ, directives, postproc_direct) = parse_circuit('./ckts/butterworth_lp.ckt')

an = parse_analysis(circ, directives)
for a in an:
  a['outfile']=tempfile.mktemp('')

res = run(circ, an_list=an)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=[12,8],dpi=100)
# plt.title(cir.title + " - TRAN Simulation")
plt.plot(res['tran']['T'], res['tran']['VN1'], label="Input voltage")
plt.plot(res['tran']['T'], res['tran']['VN4'], label="Output voltage")
plt.legend()
plt.grid(True)
# plt.ylim([0,Vin*1.5])
plt.ylabel('Step response')
plt.xlabel('Time [s]')

In [ ]:
plt.figure(figsize=[12,8],dpi=100)
plt.subplot(211)
plt.grid(True)
plt.semilogx(res['ac']['f'], np.abs(res['ac']['Vn4']), '.-')
plt.ylabel('abs(V(n4))')
plt.title("AC Simulation")
plt.grid(True)
plt.subplot(212)
plt.grid(True)
plt.semilogx(res['ac']['f'], np.angle(res['ac']['Vn4']), '.-')
plt.xlabel('Angular frequency [1/s]')
plt.ylabel('arg(V(n4)) [rad]')
plt.grid(True)

## update circuit

In [ ]:
from ahkab import time_functions 
circ.remove_elem('V1')

voltage_step = time_functions.pulse(v1=0, v2=1, td=1e-2, tr=1e-2, pw=0.5, tf=1e-12, per=2)
#voltage_step = time_functions.pulse(v1=0, v2=1, td=500e-9, tr=1e-12, pw=1, tf=1e-12, per=2)
circ.add_vsource("V1", n1="n1", n2=circ.gnd, dc_value=5, ac_value=1, function=voltage_step)

#r = ahkab.run(circ, an_list=[op_analysis, ac_analysis, tran_analysis])
`
print(circ)

## Operating Point simulation

In [ ]:
op = new_op()

r_op = run(circ, op)['op']
print(r_op)

## Transient simulation 

In [ ]:
tran = new_tran(tstart=0.0, tstop=2.25e-2, tstep=1e-3, x0=None)
r_tran = run(circ, tran)['tran']
r_tran.keys()
plt.figure(figsize=[12,8],dpi=100)
plt.hold(True)
plt.plot(r_tran['T'], r_tran['VN1'], label="Input voltage")
plt.plot(r_tran['T'], r_tran['VN4'], label="Output voltage")
plt.title(cir.title + " - TRAN Simulation")
plt.legend()
plt.grid(True)
# plt.ylim([0,Vin*1.5])
plt.ylabel('Step response')
plt.xlabel('Time [s]')